In [11]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

# Import libraries and set desired options
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy import sparse, stats
from scipy.linalg import svd
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import (KFold, StratifiedKFold, cross_val_score,
                                     cross_validate, train_test_split)
from tqdm import tqdm
from xgboost import XGBClassifier
import implicit
from scipy.sparse import csr_matrix
import scipy as sc
import datetime
import os
import skimage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 15 seconds


In [82]:
def dist(path, data_path, p, prefix):
    m = 512 // p
    files = sorted(os.listdir(path))
    images = []
    for file in files:
        full_path = os.path.join(path, file)
        image = skimage.io.imread(full_path)
        images.append(image)
    left = np.zeros((len(images), m, m, p, 3), dtype=np.uint8)
    right = np.zeros((len(images), m, m, p, 3), dtype=np.uint8)
    up = np.zeros((len(images), m, m, p, 3), dtype=np.uint8)
    down = np.zeros((len(images), m, m, p, 3), dtype=np.uint8)
    for i in range(len(images)):
        image = images[i]
        if len(image.shape) == 2:
            image = np.stack((image // 256,)*3, axis=-1)
        #print(i, image.max())
        for x in range(m):
            for y in range(m):
                left[i][x][y] = image[p * x, p * y:p * (y + 1)]
                right[i][x][y] = image[p * x + p - 1, p * y:p * (y + 1)]
                up[i][x][y] = image[p * x: p * (x + 1), p * y]
                down[i][x][y] = image[p * x: p * (x + 1), p * y + p - 1]
    ud = np.zeros((len(images), m, m, m, m))
    lr = np.zeros((len(images), m, m, m, m))
    for i in range(len(images)):
        if i % 50 == 0:
            print(i / len(images) * 100)
        for x1 in range(m):
            for y1 in range(m):
                for x2 in range(m):
                    for y2 in range(m):
                        ud[i][x1][y1][x2][y2] = np.abs(up[i][x1][y1] - down[i][x2][y2]).sum()
                        lr[i][x1][y1][x2][y2] = np.abs(left[i][x1][y1] - right[i][x2][y2]).sum()
    np.save(os.path.join(data_path, 'ud' + prefix), ud)
    np.save(os.path.join(data_path, 'lr' + prefix), lr)
dist('C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\32-sources',\
     'C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\data',
     32, '32-sources')

0.0
8.333333333333332
16.666666666666664
25.0
33.33333333333333
41.66666666666667
50.0
58.333333333333336
66.66666666666666
75.0
83.33333333333334
91.66666666666666


In [ ]:
dist('C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\32',\
     'C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\data',
     32, '32')
dist('C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\16',\
     'C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\data',
     16, '16')
dist('C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\16-sources',\
     'C:\\Users\\agano\\Documents\\notebooks\\huawei\\data_train\\data',
     16, '16-sources')

0.0
